# Rock, Paper, Scissors

#### Import modules

In [58]:
import tensorflow as tf
from tensorflow import keras
import numpy
import matplotlib.pyplot as plt
import os
import random
import sklearn
from sklearn.model_selection import StratifiedKFold


#### Default values

In [59]:
# IMPORT IMAGES
dir_images_train = "C:\\zImagens\\train"
image_size = (32,32)
batch_size = 1

# TRAINING
the_seed = 32
input_shape = image_size + (1,)
convnn_acivation = 'relu'
fully_connected_acivation = 'tanh'
hyperparameter_max = 16
learning_rate = 0.001
max_epochs = 5
score_metric = 'accuracy'
verbose = 1
phi = 360
n_folds = 10

# SET RANDOM
random.seed(the_seed)
numpy.random.seed(the_seed)
tf.random.set_seed(the_seed)

#### Import images to training dataset

In [60]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dir_images_train,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

print(train_ds)
print(train_ds.take(500))
numpy.array(train_ds)

Found 4741 files belonging to 3 classes.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_TakeDataset element_spec=(TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


array(<_BatchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
      dtype=object)

#### Training

In [61]:
def create_rps(input_size : tuple[int, int, int] = (32, 32, 1),
                      convnn_acivation : str = 'relu',
                      fully_connected_acivation : str = 'tanh',
                      net_name : str = 'RPS',
                      hyperparameter : int = 1,
                      phi = 360):
    
    filter_size = (3,3)
    pooling_size = (2,2)
    
    # CAMADA OCULTA
    inputs = tf.keras.Input(shape=input_size)
    
    # RESCALING
    scale_layer = tf.keras.layers.Rescaling(scale= 1/127.5, offset= -1)
    x = scale_layer(inputs)
    
    # CAMADA DE AUMENTO DE DADOS
    rotator = tf.keras.layers.RandomRotation(phi)
    x = rotator(x)

    # PRIMEIRA CAMADA CONVOLUCIONAL + POOL
    conv_2d_layer_1 = tf.keras.layers.Conv2D(1 * hyperparameter, filter_size, activation = convnn_acivation)
    x = conv_2d_layer_1(x)
    maxpool_1 = tf.keras.layers.MaxPooling2D(pooling_size)
    x = maxpool_1(x)

    # SEGUNDA CAMADA CONVOLUCIONAL + POOL
    conv_2d_layer_2 = tf.keras.layers.Conv2D(2 * hyperparameter, filter_size, activation = convnn_acivation)
    x = conv_2d_layer_2(x)
    maxpool_2 = tf.keras.layers.MaxPooling2D(pooling_size)
    x = maxpool_2(x)

    # TERCEIRA CAMADA CONVOLUCIONAL + POOL
    conv_2d_layer_3 = tf.keras.layers.Conv2D(4 * hyperparameter, filter_size, activation = convnn_acivation)
    x = conv_2d_layer_3(x)
    maxpool_3 = tf.keras.layers.MaxPooling2D(pooling_size)
    x = maxpool_3(x)

    # FLATTEN
    flatten_layer = tf.keras.layers.Flatten()
    x = flatten_layer(x)

    # FULLY CONNECTED HIDDEN LAYER
    hidden_layer = tf.keras.layers.Dense(64 * hyperparameter, activation = convnn_acivation)
    x = hidden_layer(x)

    # OUTPUT LAYER
    output_layer = tf.keras.layers.Dense(3, activation = fully_connected_acivation)
    outputs = output_layer(x)

    return tf.keras.Model(inputs = inputs, outputs = outputs, name = net_name)

In [62]:
result_list = []
kfolder = StratifiedKFold(n_splits = n_folds, random_state = the_seed, shuffle = True)

for hyperparameter in numpy.arange(1, hyperparameter_max + 1):
    for i, (train_index, validation_index) in enumerate(kfolder.split(numpy.array(train_ds.take(1))[0], numpy.array(train_ds.take(1))[1])):
        print(f"Starting h: {hyperparameter} f: {i}")
        X_train, Y_train = train_images[train_index], train_labels[train_index]
        X_val, Y_val = train_images[validation_index], train_labels[validation_index]
        
        # NET NAME > Hyperparameter + Fold Number
        net_name = f"LittleEyes_{hyperparameter}_{i}"
        print(f"{net_name} training started...")
        
        little_eyes = create_litle_eyes(net_name = net_name, 
                                        hyperparameter = hyperparameter)
        
        opt = optimizers.Adam(learning_rate = learning_rate)
        
        little_eyes.compile(optimizer=opt ,
                    loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=[score_metric])

        history = little_eyes.fit(X_train, Y_train, epochs = max_epochs, 
                            validation_data=(X_val, Y_val), 
                            verbose = verbose)
        
        training_score = history.history[score_metric][-1]
        validation_score = history.history[f"val_{score_metric}"][-1]
        
        result_list.append((net_name, hyperparameter, i, training_score, validation_score, little_eyes))
        print(f"{net_name.upper()}: {100 * training_score:.0f}% TRAINING {score_metric.upper()} / {100 * validation_score:.0f}% VALIDATION {score_metric.upper()}")

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed